# **Uploading the dataset**

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("combined_emails_classes.csv",encoding = "latin1") # this dataset is latin1 encoding based

In [4]:
data.head()

,email_subject,email_body,email_category,urgency level,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Account Access Issue - Invalid Credentials,I'm unable to access my {product_purchased} ac...,inquiry,medium,NaN,NaN,NaN,NaN,NaN,NaN
1,Help Needed - Action Option Missing,m having an issue with the {product_purchased}...,inquiry,medium,NaN,NaN,NaN,NaN,NaN,NaN
2,Issue Reported - Product Assistance,I'm having an issue with the {product_purchase...,inquiry,medium,NaN,NaN,NaN,NaN,NaN,NaN
3,Troubleshooting Steps Didn't Resolve Problem,I'm having an issue with the {product_purchase...,inquiry,medium,NaN,NaN,NaN,NaN,NaN,NaN
4,Concern: Product Security & Data Safety,I'm having an issue with the {product_purchase...,inquiry,medium,NaN,NaN,NaN,NaN,NaN,NaN


# **Exploratory Data Analysis**

In [5]:
data.shape

(704, 10)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   email_subject   572 non-null    object 
 1   email_body      572 non-null    object 
 2   email_category  572 non-null    object 
 3   urgency level   572 non-null    object 
 4   Unnamed: 4      0 non-null      float64
 5   Unnamed: 5      2 non-null      float64
 6   Unnamed: 6      1 non-null      float64
 7   Unnamed: 7      0 non-null      float64
 8   Unnamed: 8      0 non-null      float64
 9   Unnamed: 9      1 non-null      float64
dtypes: float64(6), object(4)
memory usage: 55.1+ KB


In [7]:
data.describe()

,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
count,0.0,2.000000,1.0,0.0,0.0,1.0
mean,NaN,394.000000,725.0,NaN,NaN,633.0
std,NaN,354.967604,NaN,NaN,NaN,NaN
min,NaN,143.000000,725.0,NaN,NaN,633.0
25%,NaN,268.500000,725.0,NaN,NaN,633.0
50%,NaN,394.000000,725.0,NaN,NaN,633.0
75%,NaN,519.500000,725.0,NaN,NaN,633.0
max,NaN,645.000000,725.0,NaN,NaN,633.0


In [8]:
data.isnull().sum() # looking for total nan values in all rows

,0
email_subject,132
email_body,132
email_category,132
urgency level,132
Unnamed: 4,704
Unnamed: 5,702
Unnamed: 6,703
Unnamed: 7,704
Unnamed: 8,704
Unnamed: 9,703


# **Data Cleaning Process**

## Drop the unwanted columns

In [9]:
data.drop(columns=["Unnamed: 4","Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Unnamed: 9"], inplace=True)

## Drop all the NaN values in all rows

In [10]:
data = data.dropna()

In [11]:
data.head()

,email_subject,email_body,email_category,urgency level
0,Account Access Issue - Invalid Credentials,I'm unable to access my {product_purchased} ac...,inquiry,medium
1,Help Needed - Action Option Missing,m having an issue with the {product_purchased}...,inquiry,medium
2,Issue Reported - Product Assistance,I'm having an issue with the {product_purchase...,inquiry,medium
3,Troubleshooting Steps Didn't Resolve Problem,I'm having an issue with the {product_purchase...,inquiry,medium
4,Concern: Product Security & Data Safety,I'm having an issue with the {product_purchase...,inquiry,medium


In [12]:
data.shape

(572, 4)

In [13]:
data.describe()

,email_subject,email_body,email_category,urgency level
count,572,572,572,572
unique,367,548,4,3
top,Payments & Transactions,Congratulations ur awarded 500 of CD vouchers ...,spam,medium
freq,43,3,151,270


In [14]:
print(data["email_category"].unique())
print(data['email_category'].value_counts())

['inquiry' 'spam' 'feedback' 'complaint']
email_category
spam         151
complaint    151
inquiry      144
feedback     126
Name: count, dtype: int64


In [15]:
print(data["urgency level"].unique())
print(data['urgency level'].value_counts())

['medium' 'low' 'high']
urgency level
medium    270
low       151
high      151
Name: count, dtype: int64


# Data Preprocessing

In [16]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


In [17]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [18]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [19]:
def clean_email(text):
    # Lowercase
    text = text.lower()
    # Remove email IDs
    text = re.sub(r'\S+@\S+', "", text)
    # Remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Tokenize
    words = text.split()
    # Remove stopwords + lemmatize
    processed_words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return " ".join(processed_words)


In [20]:
cleaned_subjects = [clean_email(e) for e in data["email_subject"]]
data["email_subject"] = cleaned_subjects

In [21]:
cleaned_body = [clean_email(e) for e in data['email_body']]
data['email_body'] = cleaned_body

In [22]:
data.head()

,email_subject,email_body,email_category,urgency level
0,account access issue invalid credential,im unable access productpurchased account keep...,inquiry,medium
1,help needed action option missing,issue productpurchased please assist possible ...,inquiry,medium
2,issue reported product assistance,im issue productpurchased please assist acknow...,inquiry,medium
3,troubleshooting step didnt resolve problem,im issue productpurchased please assist thanks...,inquiry,medium
4,concern product security data safety,im issue productpurchased please assist want a...,inquiry,medium


## Encoding the email category and urgency level

In [23]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

data['email_category'] = le.fit_transform(data['email_category'])
data['urgency level'] = le.fit_transform(data['urgency level'])

In [24]:
data.head()

,email_subject,email_body,email_category,urgency level
0,account access issue invalid credential,im unable access productpurchased account keep...,2,2
1,help needed action option missing,issue productpurchased please assist possible ...,2,2
2,issue reported product assistance,im issue productpurchased please assist acknow...,2,2
3,troubleshooting step didnt resolve problem,im issue productpurchased please assist thanks...,2,2
4,concern product security data safety,im issue productpurchased please assist want a...,2,2


In [ ]:
data.to_excel("cleaned_email_dataset.xlsx", index = False) # downloading the cleaned email data as an excel file

In [ ]:
data

,email_subject,email_body,email_category,urgency level
0,account access issue invalid credential,im unable access productpurchased account keep...,2,2
1,help needed action option missing,issue productpurchased please assist possible ...,2,2
2,issue reported product assistance,im issue productpurchased please assist acknow...,2,2
3,troubleshooting step didnt resolve problem,im issue productpurchased please assist thanks...,2,2
4,concern product security data safety,im issue productpurchased please assist want a...,2,2
...,...,...,...,...
567,telecom isp service,comcast imposed new usage cap 300gb punishes s...,0,0
568,telecom isp service,comcast working service boot,0,0
569,telecom isp service,isp charging arbitrary data limit overage fee,0,0
570,telecom isp service,throttling service unreasonable data cap,0,0


In [25]:
X = data.iloc[:,:2] # input features

y1 = data.iloc[:,2] # first output - classification
y2 = data.iloc[:,3] # second output - urgency level

In [ ]:
X

,email_subject,email_body
0,account access issue invalid credential,im unable access productpurchased account keep...
1,help needed action option missing,issue productpurchased please assist possible ...
2,issue reported product assistance,im issue productpurchased please assist acknow...
3,troubleshooting step didnt resolve problem,im issue productpurchased please assist thanks...
4,concern product security data safety,im issue productpurchased please assist want a...
...,...,...
567,telecom isp service,comcast imposed new usage cap 300gb punishes s...
568,telecom isp service,comcast working service boot
569,telecom isp service,isp charging arbitrary data limit overage fee
570,telecom isp service,throttling service unreasonable data cap


In [ ]:
y1

,email_category
0,2
1,2
2,2
3,2
4,2
...,...
567,0
568,0
569,0
570,0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# combine two features into one string per row
combined_text = data['email_subject'].fillna("") + " " + data['email_subject'].fillna("")

# single string row will be given as as input to TfidVectorizer
vectorizer = TfidfVectorizer(max_features  = 5000,
                             min_df = 7,
                             max_df = 0.2,
                             ngram_range = (1,2))

X = vectorizer.fit_transform(combined_text).toarray()


In [ ]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Splitting into training and testing

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y1, test_size = 0.2, random_state = 42) # for classifying the email categories

# Logistic Regression

A common issue in machine learning is receiving a ConvergenceWarning, which means the model stopped training because it hit the max_iter limit before finding the optimal solution. In such cases, increasing the number of iterations can resolve the warning and potentially improve the model's performance

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter = 1000)
lr.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [ ]:
y1_pred = lr.predict(X_test)
y1_pred

array([0, 2, 2, 3, 0, 1, 3, 2, 2, 1, 0, 3, 0, 0, 0, 2, 1, 3, 2, 3, 2, 3,
       1, 3, 0, 2, 3, 1, 3, 2, 3, 3, 0, 3, 3, 2, 2, 3, 2, 2, 2, 0, 1, 3,
       3, 2, 3, 0, 3, 0, 1, 2, 3, 3, 3, 1, 3, 2, 2, 3, 2, 2, 3, 3, 3, 0,
       2, 2, 3, 3, 1, 2, 1, 0, 1, 0, 3, 1, 3, 3, 2, 3, 2, 1, 3, 0, 2, 0,
       3, 0, 2, 3, 1, 0, 1, 2, 3, 3, 3, 3, 0, 2, 2, 1, 0, 1, 0, 3, 0, 3,
       2, 3, 0, 0, 2])

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

accuracy = accuracy_score(y_test,y1_pred)
print("Accuracy:",(accuracy)*100)

Accuracy: 90.0


In [ ]:
report = classification_report(y_test,y1_pred)
print("Report:",report)

Report:               precision    recall  f1-score   support

           0       1.00      0.80      0.89        30
           1       0.88      0.79      0.83        19
           2       1.00      1.00      1.00        32
           3       0.79      0.97      0.87        34

    accuracy                           0.90       115
   macro avg       0.92      0.89      0.90       115
weighted avg       0.92      0.90      0.90       115



# Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train,y_train)

MultinomialNB()

In [ ]:
y1_pred2 = nb.predict(X_test)
y1_pred2

array([0, 2, 2, 0, 0, 1, 0, 2, 2, 1, 0, 0, 0, 0, 0, 2, 1, 0, 2, 0, 2, 3,
       1, 0, 0, 2, 3, 1, 3, 2, 0, 3, 0, 0, 3, 2, 2, 0, 2, 2, 2, 0, 1, 3,
       3, 2, 3, 0, 3, 0, 1, 2, 3, 3, 0, 1, 0, 2, 2, 0, 2, 2, 3, 0, 3, 0,
       2, 2, 0, 3, 1, 2, 1, 0, 1, 0, 0, 1, 0, 0, 2, 3, 2, 1, 3, 0, 2, 0,
       3, 0, 2, 3, 1, 0, 1, 2, 3, 3, 3, 3, 0, 2, 2, 1, 0, 1, 0, 0, 0, 3,
       2, 3, 0, 0, 2])

In [ ]:
accuracy = accuracy_score(y_test,y1_pred2)
print("Accuracy:",round(accuracy,2)*100)

Accuracy: 82.0


In [ ]:
report = classification_report(y_test,y1_pred2)
print("Report:",report)

Report:               precision    recall  f1-score   support

           0       0.62      0.87      0.72        30
           1       0.88      0.79      0.83        19
           2       1.00      1.00      1.00        32
           3       0.88      0.62      0.72        34

    accuracy                           0.82       115
   macro avg       0.84      0.82      0.82       115
weighted avg       0.84      0.82      0.82       115



# Fine tuning using Transformer models

## Using DistilBERT Model


In [ ]:
import torch
from transformers import DistilBertTokenizerFast,DistilBertForSequenceClassification

## converting combined texts and labels into list for BERT abd DistilBERT tokenization


In [ ]:
email_list = combined_text.tolist()
category_list = y1.tolist()

In [ ]:
email_list[1:5]

['help needed action option missing help needed action option missing',
 'issue reported product assistance issue reported product assistance',
 'troubleshooting step didnt resolve problem troubleshooting step didnt resolve problem',
 'concern product security data safety concern product security data safety']

## Tokenizer

In [ ]:

tokenizer = DistilBertTokenizerFast.from_pretrained(
    "distilbert-base-uncased"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

## Define the Torch Dataset

When you fine‑tune a transformer model (like DistilBERT), the Trainer or a PyTorch DataLoader expects the data in a very specific format:
- Inputs: tokenized tensors (input_ids, attention_mask, etc.)
- Labels: numeric class IDs (0, 1, 2, 3 for your categories)


Your raw data (texts, labels) is just Python lists. The tokenizer gives you NumPy arrays or lists of IDs. But the model needs PyTorch tensors wrapped in a dataset object that can be indexed and iterated.


In [ ]:
class EmailDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels): # stores the encodings (tokenized emails) and category labels
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx): # when you ask for dataset[5], it returns a dictionary like
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self): # tells Pytorch how many samples are in the dataset
    return len(self.labels)

## Splitting into training and testing


In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    email_list, category_list, test_size = 0.2, random_state = 42
)

In [ ]:
# tokenize the train and test emails

train_encodings = tokenizer(
    train_texts,
    truncation=True, # cut the long emails
    padding=True, # equal length for batching
    max_length=128)

test_encodings = tokenizer(
    test_texts,
    truncation=True,
    padding=True,
    max_length=128)


In [ ]:
# Make them as Pytorch Dataset

train_dataset = EmailDataset(train_encodings, train_labels)
test_dataset = EmailDataset(test_encodings, test_labels)


In [ ]:
# Load the DistilBERT model

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels = 4 # 4 email category
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Model Training

In [ ]:

from transformers import Trainer,TrainingArguments

training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 3, # finetuning
    per_device_train_batch_size = 8, # safe for limited GPU
    logging_dir = "./logs",
    logging_steps = 10,
    save_strategy = "no" # avoids the saving checkpoints for the demo
)

In [ ]:
trainer1 = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
)

trainer1.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.065700
20,0.170100
30,0.002800
40,0.001400
50,0.000700
60,0.000600
70,0.014200
80,0.001400
90,0.000400
100,0.046700


TrainOutput(global_step=174, training_loss=0.017626491497035258, metrics={'train_runtime': 462.137, 'train_samples_per_second': 2.967, 'train_steps_per_second': 0.377, 'total_flos': 21136625069040.0, 'train_loss': 0.017626491497035258, 'epoch': 3.0})

## Evaluation

In [ ]:
predictions = trainer1.predict(test_dataset)
y_pred = predictions.predictions.argmax(axis=1)
y_true = predictions.label_ids


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
acc = accuracy_score(y_true, y_pred)
print("Accuracy:", acc*100)

report = classification_report(y_true,y_pred)
print("Classification report:", report)

Accuracy: 97.3913043478261
Classification report:               precision    recall  f1-score   support

           0       1.00      0.93      0.97        30
           1       0.95      1.00      0.97        19
           2       0.94      1.00      0.97        32
           3       1.00      0.97      0.99        34

    accuracy                           0.97       115
   macro avg       0.97      0.98      0.97       115
weighted avg       0.98      0.97      0.97       115



## Using BERT model

In [ ]:
from transformers import BertTokenizerFast, BertForSequenceClassification

train_texts, test_texts, train_labels, test_labels = train_test_split(
    email_list, category_list, test_size=0.2, random_state=42
)


### Tokenizer

In [ ]:
# Tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


In [ ]:
# ---------------- Dataset Class ----------------
class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EmailDataset(train_encodings, train_labels)
test_dataset = EmailDataset(test_encodings, test_labels)


In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=4  # compliant, request, spam, feedback
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args1 = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="no"
)


In [ ]:
trainer2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer2.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,1.244800
20,0.826400
30,0.623100
40,0.389600
50,0.271200
60,0.111400
70,0.086100
80,0.116800
90,0.080300
100,0.115600


TrainOutput(global_step=174, training_loss=0.23061437880213576, metrics={'train_runtime': 440.4024, 'train_samples_per_second': 3.113, 'train_steps_per_second': 0.395, 'total_flos': 21136625069040.0, 'train_loss': 0.23061437880213576, 'epoch': 3.0})

In [ ]:
predictions = trainer2.predict(test_dataset)
y_pred = predictions.predictions.argmax(axis=1)
y_true = predictions.label_ids

print("Accuracy:", accuracy_score(y_true, y_pred)*100)


print("Classification Report:",classification_report(y_true, y_pred))


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Accuracy: 95.65217391304348
Classification Report:               precision    recall  f1-score   support

           0       1.00      0.93      0.97        30
           1       0.95      1.00      0.97        19
           2       1.00      0.94      0.97        32
           3       0.89      0.97      0.93        34

    accuracy                           0.96       115
   macro avg       0.96      0.96      0.96       115
weighted avg       0.96      0.96      0.96       115



# Saving the best model for the classification for local deployment


DistilBERT model has the highest accuracy

In [ ]:
# Save the trained model
trainer1.model.save_pretrained("saved_model")

# Save the tokenizer (must be the same one you used for training)
tokenizer.save_pretrained("saved_model")

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.txt',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')

In [ ]:
import os
print(os.listdir("saved_model"))

['model.safetensors', 'vocab.txt', 'tokenizer.json', 'config.json', 'tokenizer_config.json', 'special_tokens_map.json']
